In [ ]:
import numpy
import json
import numpy as np

from scipy import sparse
from scipy.stats import hypergeom
from scipy import spatial

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

from bluepysnap import Circuit

In [ ]:
circuit_path = '/home/data-bbp/20191017/circuit_config.json'
circuit = Circuit(circuit_path)
cells = circuit.nodes["hippocampus_neurons"]
conn = circuit.edges["hippocampus_neurons__hippocampus_neurons__chemical"]

analyze_population = 'Excitatory'
n_smpl = 2500
analyze_gids = cells.ids(analyze_population, sample=n_smpl)  # get the identifiers of target neurons
print(len(analyze_gids))

In [ ]:
def efferent_con_mat(pop_gids):
    '''Returns a sparse matrix of the EFFERENT connectivity of neurons in
    the specified population'''
    shape = (len(pop_gids), cells.size) # The output shape is (number of neurons in population x number of neurons in circuit)
    post = [conn.efferent_nodes(_g) for _g in pop_gids]  # Get identifiers of connected neurons
    
    '''prepare the sparse matrix representation, where the column indices for row i are stored in
    "indices[indptr[i]:indptr[i+1]]" and their corresponding values are stored in "data[indptr[i]:indptr[i+1]]".'''
    indptr = numpy.hstack((0, numpy.cumsum(list(map(len, post)))))
    indices = numpy.hstack(post)
    data = numpy.ones_like(indices, dtype=bool) # Simple boolean connection matrix. A connection exists or not.
    return sparse.csr_matrix((data, indices, indptr), shape=shape)

#For this example, we will look at connectivity and common neighbors between L5_MC neurons.
connections = efferent_con_mat(analyze_gids)

In [ ]:
# Let's look at the result
vmin = np.min(connections) + 0.01   # +0.01 to avoid log(0) inside the plot
vmax = np.max(connections)

ax = plt.figure().add_axes([0.1, 0.1, 0.8, 0.8])
ax.imshow(connections.toarray()[:1000, 9000:10000], cmap='Reds', norm=LogNorm(vmin=vmin, vmax=vmax))

In [ ]:
def common_efferent_neighbors(M):
    CN = M.astype(int) * M.astype(int).transpose() # need to convert to int, so that neighbors are counted
    return CN.toarray()

com_neighs = common_efferent_neighbors(connections)
vmin = np.min(com_neighs) + 0.01   # +0.01 to avoid log(0) inside the plot
vmax = np.max(com_neighs)

ax = plt.figure().add_axes([0.1, 0.1, 0.8, 0.8])
ax.imshow(com_neighs, cmap='Reds', norm=LogNorm(vmin=vmin, vmax=vmax));

In [ ]:
def cn_mat_to_histogram(CN, bins):
    '''limit to upper triangular matrix. This excludes the diagonal entries and redundant entries, because the common
    neighbor matrix is always symmetrical!
    '''
    return numpy.histogram(numpy.triu(CN, 1), bins=bins)[0]

xbins = numpy.arange(502) # count from 0 to 500 common neighbors
H = cn_mat_to_histogram(com_neighs, xbins)

In [ ]:
def control_erdos_renyi_histogram(CN, N, bins):
    out_degrees = numpy.diag(CN)
    '''Note: Here, we simply draw a random sample for each pair of neurons.
    Better, but more expensive would be to evaluate the probability mass function
    for all bins and for all pairs.'''
    expected = [hypergeom(N, d_A, out_degrees[(i+1):]).rvs()
                for i, d_A in enumerate(out_degrees)]
    return numpy.histogram(numpy.hstack(expected), bins=bins)[0]

H_ctrl_er = control_erdos_renyi_histogram(com_neighs, connections.shape[1], xbins)

ax = plt.figure().add_axes([0.1, 0.1, 0.8, 0.8])
ax.plot(xbins[:-1], H, color='red', marker='o', label='Experiment')
ax.plot(xbins[:-1], H_ctrl_er, color='black', marker='o', label='Control (ER)')
ax.set_yscale('log'); ax.legend(); ax.set_xlabel('Common neighbors'); ax.set_ylabel('Pairs')


In [ ]:
nbins = 50

def connect_keep_dist_dep(D, C, nbins):
    '''Randomly connect a number of neurons, keeping their distance dependence intact.
    D: Matrix (AxN) of distances to all other neurons in the circuit
    C: Matrix (AxN) of connections to all other neurons in the circuit (boolean)'''
    dbins = numpy.linspace(0, D.max(), nbins + 1) + 0.1
    Di = numpy.digitize(D, bins=dbins) - 1
    H_connected = numpy.histogram(Di[C.toarray()], bins=range(nbins + 1))[0]
    H_all = numpy.histogram(Di, bins=range(nbins + 1))[0]
    P = H_connected.astype(float) / H_all
    n_eff = numpy.array(C.sum(axis=1)).transpose()[0]
    indptr = [0]
    indices = []
    for row, n in zip(Di, n_eff):
        p_row = P[row]
        p_row[row == -1] = 0
        p_row = p_row / p_row.sum()
        rnd = numpy.random.choice(len(row), n, replace=False, p=p_row)
        indices.extend(rnd)
        indptr.append(indptr[-1] + n)
    data = numpy.ones_like(indices, dtype=bool)
    return sparse.csr_matrix((data, indices, indptr), shape=D.shape)

xyz = cells.positions()
# get distance matrix as an input
D = spatial.distance_matrix(xyz.loc[analyze_gids], xyz)

# generate random instance
rnd_connections = connect_keep_dist_dep(D, connections, nbins)

# compare distribution of distances of connected neurons
ax = plt.figure().add_axes([0.1, 0.1, 0.8, 0.8])
ax.plot(sorted(D[connections.toarray()]), label='Data')
ax.plot(sorted(D[rnd_connections.toarray()]), label='Control')
ax.legend(); ax.set_xlabel('Connection #'); ax.set_ylabel('Distance (um)');

In [ ]:
rnd_com_neighs = common_efferent_neighbors(rnd_connections)
rnd_H = cn_mat_to_histogram(rnd_com_neighs, xbins)

ax = plt.figure().add_axes([0.1, 0.1, 0.8, 0.8])
ax.plot(xbins[:-1], H, color='red', marker='o', label='Experiment')
ax.plot(xbins[:-1], rnd_H, color='black', marker='o', label='Control (Dist-dep.)')
ax.set_yscale('log'); ax.legend(); ax.set_xlabel('Common neighbors'); ax.set_ylabel('Pairs');

In [ ]:
def cn_bias_1(H_data, H_ctrl):
    assert len(H_data) == len(H_ctrl)
    log_data = numpy.log10(H_data[1:]) # exclude the bin at 0
    log_ctrl = numpy.log10(H_ctrl[1:])
    idx_data = numpy.nonzero(~numpy.isinf(log_data))[0] # exclude zero bins
    idx_ctrl = numpy.nonzero(~numpy.isinf(log_ctrl))[0]
    slope_data = numpy.polyfit(idx_data.astype(float), log_data[idx_data], 1)[0]
    slope_ctrl = numpy.polyfit(idx_ctrl.astype(float), log_ctrl[idx_ctrl], 1)[0]
    return (slope_ctrl - slope_data) / slope_ctrl

print(cn_bias_1(H, rnd_H))